### 简历项目2：使用QLoRA+AWQ处理基于LLaMA3.1-8B的智能客服

#### 1. 数据集
&emsp;&emsp; **E_commerce_Customer_Service_dataset** 可以从魔搭社区下载，电商客户服务数据集，核心定位是为电商客服机器人的研究提供数据支持。它包含从lightinthebox网站收集的电商数据，包括通用问题的问答对以及商品信息，**样本展示**<br>
&emsp;&emsp;{"messages": [{"role": "system", "content": "你是一个专业的电商客服。严格遵守：1) 仅以‘assistant’角色回答一次；2) 不要提出新问题，不要延长对话；3) 不要生成‘user’或其他角色的内容；4) 输出只包含答案正文，无前后缀角色标签、无思考过程。"}, {"role": "user", "content": "那要怎样这也不行那也不行"}, {"role": "assistant", "content": "发邮政的哦"}]} <br>
&emsp;&emsp;{"messages": [{"role": "system", "content": "你是一个专业的电商客服。严格遵守：1) 仅以‘assistant’角色回答一次；2) 不要提出新问题，不要延长对话；3) 不要生成‘user’或其他角色的内容；4) 输出只包含答案正文，无前后缀角色标签、无思考过程。"}, {"role": "user", "content": "可以优惠价么"}, {"role": "assistant", "content": "我们因为200把起给优惠价格您可以给您105元一把哦"}]}<br>
&emsp;&emsp;{"messages": [{"role": "system", "content": "你是一个专业的电商客服。严格遵守：1) 仅以‘assistant’角色回答一次；2) 不要提出新问题，不要延长对话；3) 不要生成‘user’或其他角色的内容；4) 输出只包含答案正文，无前后缀角色标签、无思考过程。"}, {"role": "user", "content": "我去不早说发韵达能到我家那儿我就能拿到"}, {"role": "assistant", "content": "韵达不发的哦"}]}<br>
#### 2. SFT阶段

#### 3. AWQ量化阶段
